In [1]:
import torch
from brt.common import log
import brt.nn as nn
from brt.router import RandomScatterRouter, GatherRouter


class MoE(nn.Module):
    def __init__(self):
        super().__init__()
        self.scatter_router = RandomScatterRouter(route_num=2)
        self.expert1 = nn.Linear(10, 10)
        self.expert2 = nn.Linear(10, 10)
        self.gather_router = GatherRouter(route_num=2)

    def forward(self, x):
        route_results, route_tags, loads = self.scatter_router(x)
        x_0 = self.expert1(route_results[0])
        x_1 = self.expert2(route_results[1])
        x = self.gather_router([x_0, x_1], route_tags, loads)
        return x


class ThorMoE(nn.Module):
    def __init__(
        self,
    ):
        super().__init__()
        self.moe = MoE()

    def forward(self, x):
        return self.moe(x)


thor_moe = ThorMoE()
x = torch.arange(0, 30, dtype=torch.float32).view(3, 10)
x = thor_moe(x)
print(x)


tensor([[ -0.1637,   1.9598,  -1.8768,   0.4871,   0.3627,  -1.6415,  -2.2952,
          -4.3027,   0.3830,  -0.3647],
        [ -1.1561,   7.7347,   2.5445,  14.5481, -13.4185,  13.1255,   1.7102,
          -8.8952, -13.6887, -11.6499],
        [  1.3115,  10.0178,   3.5990,   3.9877,   2.2013, -14.4831, -11.1553,
         -21.3098,  -7.2553,   1.8244]], grad_fn=<ScatterAddBackward0>)
